In [1]:
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.1 MB/s eta 0:00:00


In [2]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [3]:
!pip uninstall -y transformers

Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.0 MB/s eta 0:00:00


In [5]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

numpy       : 1.26.4
pandas      : 2.2.2
torch       : 2.5.1+cu118
transformers: 4.48.1



In [6]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import random
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss



##Trainer от HuggingFace

In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset

imdb_dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
imdb_dataset["train"]["text"][:10]

['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [10]:
num_cls = len(set(imdb_dataset["train"]["label"]))

In [14]:
set(imdb_dataset["train"]["label"])

{0, 1}

In [13]:
num_cls

2

Сократим количество примеров, чтобы процесс обучения был быстрее.

In [15]:
train_texts = imdb_dataset["train"]["text"][:2000]
train_labels = imdb_dataset["train"]["label"][:2000]
test_texts = imdb_dataset["test"]["text"][:1000]
test_labels = imdb_dataset["test"]["label"][:1000]

И в трейне и в тесте первые несколько тысяч примеров, которые и попадали в используемые выборки, были отрицательными, то есть модель и на обучении, и на валидации, и на тесте видела только примеры с меткой 0((.

In [11]:
train_size = len(imdb_dataset["train"]["text"]) #25000
test_size = len(imdb_dataset["test"]["text"]) #25000

train_indices = np.random.choice(train_size, 2000, replace=False) #берем рандомные числа для индексов объектов, которые попадут в выборки
test_indices = np.random.choice(test_size, 1000, replace=False)

train_texts = [imdb_dataset["train"]["text"][i] for i in train_indices] #по индексам выбираем случайные объекты из всего датасета
train_labels = [imdb_dataset["train"]["label"][i] for i in train_indices]

test_texts = [imdb_dataset["test"]["text"][i] for i in test_indices]
test_labels = [imdb_dataset["test"]["label"][i] for i in test_indices]

In [12]:
del imdb_dataset

Выделим часть обучающих данных для оценки:

In [13]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [14]:
print(val_labels)

[0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 

In [15]:
print(train_labels)

[0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 

In [16]:
print(test_labels)

[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 

Когда проверила то же самое с первым вариантом выбора объектов (первые n объектов подряд), были одни нули. Почему-то не подумала с первого раза распределение меток проверить(

Токенизируем данные

In [17]:
from transformers import AutoTokenizer, AutoModel
PRE_TRAINED_MODEL_NAME = "albert/albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Создадим класс Dataset, который на каждой итерации будет возвращать токенизированный отзыв и класс:

In [19]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Опишем функцию для подсчета метрик, которые хотим увидеть при оценке модели (по умолчанию будет вычисляться только loss):

In [20]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions


    preds = torch.tensor(preds).cpu()
    labels = torch.tensor(labels).cpu()

    predictions = torch.argmax(preds, axis=-1)

    predictions = predictions.numpy()
    labels = labels.numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Зададим параметры обучения с помощью TrainingArguments:

In [21]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [22]:
device

device(type='cuda', index=0)

In [23]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

Инициализируем модель и запустим fine-tune ее с помощью модуля Trainer. В данном модуле будет автоматически происходить итерация по эпохам, батчам, рассчет функции потерь и градиентов, ничего этого больше руками прописывать не нужно!

In [24]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask, labels=None):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=False
    )
    pooled_output = outputs[1] #только выход с пуллер-слоя
    logits = self.out(self.drop(pooled_output))

    loss = None
    if labels is not None:
        loss_fn = CrossEntropyLoss()
        loss = loss_fn(logits, labels)

    return {"loss": loss, "logits": logits}

In [25]:
model = SentimentClassifier(num_cls).to(device)


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,0.817000
20,0.804200
30,0.692200
40,0.722700
50,0.656400
60,0.720200
70,0.667300
80,0.628500
90,0.526900
100,0.473800


TrainOutput(global_step=400, training_loss=0.4752315300703049, metrics={'train_runtime': 316.206, 'train_samples_per_second': 10.12, 'train_steps_per_second': 1.265, 'total_flos': 0.0, 'train_loss': 0.4752315300703049, 'epoch': 2.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.3643721044063568,
 'eval_accuracy': 0.91,
 'eval_f1': 0.9146919431279621,
 'eval_precision': 0.8772727272727273,
 'eval_recall': 0.9554455445544554,
 'eval_runtime': 12.6338,
 'eval_samples_per_second': 31.661,
 'eval_steps_per_second': 1.979,
 'epoch': 2.0}

Оценим модель на тестовых данных

In [28]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

{'test_loss': 0.5049291849136353,
 'test_accuracy': 0.869,
 'test_f1': 0.875829383886256,
 'test_precision': 0.8279569892473119,
 'test_recall': 0.9295774647887324,
 'test_runtime': 31.5467,
 'test_samples_per_second': 31.699,
 'test_steps_per_second': 1.997,
 'epoch': 2.0}

Выход с пуллер-слоя + эмбеддинг cls-токена с последнего слоя:

In [32]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size * 2, n_classes)  #увеличила входной размер, т.к. будем конкатенировать выходы с пуллер-слоя и с последнего

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )


        pooled_output = outputs[1]  #выход с пуллер слоя


        cls_embedding = outputs[0][:, 0, :]  #cls-токены (как я поняла, можно еще было сделать return_dict=True, cls_embedding = outputs['last_hidden_state'][:, 0, :])


        combined_output = torch.cat((pooled_output, cls_embedding), dim=1)  #конкатенируем, получаем матрицу [batch_size, hidden_size*2]


        logits = self.out(self.drop(combined_output))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

In [33]:
model2 = SentimentClassifier(num_cls)


In [34]:
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
10,0.740100
20,0.911700
30,0.856800
40,0.735300
50,0.747700
60,0.711500
70,0.709100
80,0.736700
90,0.583700
100,0.543100


TrainOutput(global_step=400, training_loss=0.5170657867193222, metrics={'train_runtime': 301.1666, 'train_samples_per_second': 10.625, 'train_steps_per_second': 1.328, 'total_flos': 0.0, 'train_loss': 0.5170657867193222, 'epoch': 2.0})

In [35]:
trainer.evaluate()

{'eval_loss': 0.7667707800865173,
 'eval_accuracy': 0.7975,
 'eval_f1': 0.7552870090634441,
 'eval_precision': 0.9689922480620154,
 'eval_recall': 0.6188118811881188,
 'eval_runtime': 12.7135,
 'eval_samples_per_second': 31.463,
 'eval_steps_per_second': 1.966,
 'epoch': 2.0}

In [36]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

{'test_loss': 0.9650571942329407,
 'test_accuracy': 0.762,
 'test_f1': 0.6932989690721649,
 'test_precision': 0.96415770609319,
 'test_recall': 0.5412474849094567,
 'test_runtime': 31.5535,
 'test_samples_per_second': 31.692,
 'test_steps_per_second': 1.997,
 'epoch': 2.0}

cls-токены с нескольких слоев:  
В этот раз решила не конкатенировать векторы, а брать средний из всех cls, чтобы он не поличился слишком огромный

In [37]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes, selected_layers=None, dropout=0.3):
        super().__init__()
        self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME, output_hidden_states=True)
        self.selected_layers = selected_layers if selected_layers else [-1]   #если модель не получит кол-во слоев, с которых хотим брать cls, она возьмет их только с последнего
        self.hidden_size = self.bert.config.hidden_size
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(self.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        hidden_states = outputs.hidden_states  #список скрытых состояний

        cls_tokens = [hidden_states[i][:, 0, :] for i in self.selected_layers]  #cls из скрытых состояний

        averaged_cls = torch.mean(torch.stack(cls_tokens), dim=0) #усредняем

        logits = self.out(self.drop(averaged_cls))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

In [38]:
LAYERS = [0, 4, 7, 11] #сразу со всех решила не собирать

In [39]:
model3 = SentimentClassifier(num_cls, LAYERS)

In [40]:
trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
10,0.755100
20,0.684700
30,0.717400
40,0.686200
50,0.674100
60,0.730000
70,0.703200
80,0.643400
90,0.613900
100,0.516100


TrainOutput(global_step=400, training_loss=0.47230651542544366, metrics={'train_runtime': 285.3936, 'train_samples_per_second': 11.213, 'train_steps_per_second': 1.402, 'total_flos': 0.0, 'train_loss': 0.47230651542544366, 'epoch': 2.0})

In [41]:
trainer.evaluate()

{'eval_loss': 0.38411790132522583,
 'eval_accuracy': 0.8775,
 'eval_f1': 0.870026525198939,
 'eval_precision': 0.9371428571428572,
 'eval_recall': 0.8118811881188119,
 'eval_runtime': 12.614,
 'eval_samples_per_second': 31.711,
 'eval_steps_per_second': 1.982,
 'epoch': 2.0}

In [42]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

{'test_loss': 0.45845502614974976,
 'test_accuracy': 0.86,
 'test_f1': 0.8491379310344828,
 'test_precision': 0.91415313225058,
 'test_recall': 0.7927565392354124,
 'test_runtime': 31.5677,
 'test_samples_per_second': 31.678,
 'test_steps_per_second': 1.996,
 'epoch': 2.0}

In [43]:
LAYERS = list(range(0, 12)) #все слои

In [44]:
model3 = SentimentClassifier(num_cls, LAYERS)

In [45]:
trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
10,0.752800
20,0.687500
30,0.728100
40,0.687100
50,0.685900
60,0.748900
70,0.713800
80,0.653600
90,0.648500
100,0.589300


TrainOutput(global_step=400, training_loss=0.5097958797216415, metrics={'train_runtime': 286.5434, 'train_samples_per_second': 11.168, 'train_steps_per_second': 1.396, 'total_flos': 0.0, 'train_loss': 0.5097958797216415, 'epoch': 2.0})

In [46]:
trainer.evaluate()

{'eval_loss': 0.407912015914917,
 'eval_accuracy': 0.8875,
 'eval_f1': 0.8883374689826302,
 'eval_precision': 0.8905472636815921,
 'eval_recall': 0.8861386138613861,
 'eval_runtime': 12.6203,
 'eval_samples_per_second': 31.695,
 'eval_steps_per_second': 1.981,
 'epoch': 2.0}

In [47]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

{'test_loss': 0.5448837876319885,
 'test_accuracy': 0.855,
 'test_f1': 0.8568608094768015,
 'test_precision': 0.8410852713178295,
 'test_recall': 0.8732394366197183,
 'test_runtime': 31.5425,
 'test_samples_per_second': 31.703,
 'test_steps_per_second': 1.997,
 'epoch': 2.0}

Применение готовой модели:

In [48]:
from transformers import AutoModelForSequenceClassification
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model4 = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
trainer = Trainer(
    model=model4,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
10,1.582200
20,0.819700
30,0.829700
40,0.712800
50,0.741600
60,0.836200
70,0.707300
80,0.839500
90,0.702800
100,0.706000


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

TrainOutput(global_step=400, training_loss=0.7459525752067566, metrics={'train_runtime': 312.8445, 'train_samples_per_second': 10.229, 'train_steps_per_second': 1.279, 'total_flos': 841962936729600.0, 'train_loss': 0.7459525752067566, 'epoch': 2.0})

In [50]:
trainer.evaluate()

{'eval_loss': 0.6870836019515991,
 'eval_accuracy': 0.4975,
 'eval_f1': 0.028985507246376812,
 'eval_precision': 0.6,
 'eval_recall': 0.01485148514851485,
 'eval_runtime': 11.0905,
 'eval_samples_per_second': 36.067,
 'eval_steps_per_second': 2.254,
 'epoch': 2.0}

In [51]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

{'test_loss': 0.6896779537200928,
 'test_accuracy': 0.514,
 'test_f1': 0.04330708661417323,
 'test_precision': 1.0,
 'test_recall': 0.022132796780684104,
 'test_runtime': 28.0675,
 'test_samples_per_second': 35.628,
 'test_steps_per_second': 2.245,
 'epoch': 2.0}

На готовой модели качество заметно хуже получилось.  
Самая высокая accuracy на тесте в итоге у первой модели (0.869, хотя у остальных совсем чуть-чуть пониже). Худший результат (не считая готовую модель) у модели, в которой использовались cls токены с последнего слоя.

Проверка на отзывах (взяла с imdb, отрицательный, положительный, отрицательный), вторая модель (выходы с пуллер слоя+cls токены):

In [29]:
reviews = ["This movie was disappointing to me because it had some really nice things going which made it promising for the first 66%. In the end however, it dragged to an over-the top disastrous climax which was really more gross than clever or insightful. I wish the end of the movie had been rewritten to perhaps have a confrontation with the drug makers rather than be all about her demise at a big public media event.", "I just watched one of the best films of the year. The Athens festival nights have kicked off, and at the opening ceremony, we were lucky enough to catch Anora. And let me tell you, this film absolutely slays.", "-Bad acting: a golden globe nomination and probably an Oscar-nomination too for the actress?!!!! FOR WHAT?!! "]
class_names = ['negative', 'positive']

In [31]:
for review in reviews:
    encoded_review = tokenizer.encode_plus(
        review,
        max_length=160,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    logits = output['logits']
    prediction = torch.argmax(logits, dim=1)

    print(f'Review text: {review}')
    print(f'Sentiment  : {class_names[prediction]}')

Review text: This movie was disappointing to me because it had some really nice things going which made it promising for the first 66%. In the end however, it dragged to an over-the top disastrous climax which was really more gross than clever or insightful. I wish the end of the movie had been rewritten to perhaps have a confrontation with the drug makers rather than be all about her demise at a big public media event.
Sentiment  : negative
Review text: I just watched one of the best films of the year. The Athens festival nights have kicked off, and at the opening ceremony, we were lucky enough to catch Anora. And let me tell you, this film absolutely slays.
Sentiment  : positive
Review text: -Bad acting: a golden globe nomination and probably an Oscar-nomination too for the actress?!!!! FOR WHAT?!! 
Sentiment  : negative


Все правильно сделал!